*#MARS2020*

##### Helpers

In [ ]:
#MARS
#Create a clothing image classification network using fashion_mnist data set
#data set contains 70,000 images divided into 10 categories
#60,000 of these images will be fed into the training model while 10,000 will be used to test model accuracy

# TensorFlow and tf.keras
import tensorflow.keras as tfk
import numpy as np
import random
import pandas as pd
import matplotlib.pyplot as plt

plt.rcParams["savefig.format"] = 'png'

pd.set_option("display.max_rows", None, "display.max_columns", None)

fashion_mnist = tfk.datasets.fashion_mnist

(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

#Normalize the value of images
train_images = train_images / 255.0
test_images = test_images / 255.0

(x_train, x_valid) = train_images[10000:], train_images[:10000]
(y_train, y_valid) = train_labels[10000:], train_labels[:10000]

class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

data_check = pd.read_csv('Fashion\check.csv')

##### Assigning Values

In [ ]:
n_hidden_layers = 1
n_neurons = 256
batch_size = 100
n_epochs = 5

##### Model Creation

In [ ]:
params = str(n_hidden_layers)+' hidden layer(s); '+str(n_neurons)+' neuron(s); '+str(batch_size)+' sample(s)/batch; '+str(n_epochs)+' epoch(s)'

#MAIN MODEL FUNCTION
#Building the Neural Network Architecture
model = tfk.Sequential()
model.add(tfk.layers.Flatten(input_shape=(28, 28)))
for z in range(n_hidden_layers):
    model.add(tfk.layers.Dense(n_neurons, activation='relu'))
model.add(tfk.layers.Dense(10))

model.compile(optimizer='adam', 
              loss=tfk.losses.SparseCategoricalCrossentropy(from_logits=True), 
              metrics=['accuracy'])
        
#Training the Model
history = model.fit(x_train, y_train, batch_size=batch_size, epochs=n_epochs, validation_data=(x_valid,  y_valid))
train_acc = history.history['accuracy']
train_loss = history.history['loss']
valid_acc = history.history['val_accuracy']
valid_loss = history.history['val_loss']
x_axis = list(range(1, n_epochs+1))
#Plotting model accuracy and loss        
def labels(y_array):
    for x,y in zip(x_axis, y_array):
        label = "{:.2f}".format(y)
        plt.annotate(label, (x,y), textcoords="offset points", xytext=(0,5), ha='center')
plt.figure(figsize=(15,7)).patch.set_facecolor('white')
plt.plot(x_axis, train_acc, marker='o')
labels(train_acc)
plt.plot(x_axis, valid_acc, marker='o')
labels(valid_acc)
plt.title('MODEL ACCURACY\n\n'+params)
plt.xticks(x_axis)
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['training', 'validation'], loc='upper left')
plt.show()
plt.figure(figsize=(15,7)).patch.set_facecolor('white')
plt.plot(x_axis, train_loss, marker='o')
labels(train_loss)
plt.plot(x_axis, valid_loss, marker='o')
labels(valid_loss)
plt.title('MODEL LOSS\n\n'+params)
plt.xticks(x_axis)
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['training', 'validation'], loc='upper left')
#plt.savefig("Model History", dpi=300)
plt.show()

#Evaluate Training Accuracy
print("Test 10000 images: ")
test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)

#Making Predictions
probability_model = tfk.Sequential([model, tfk.layers.Softmax()])
predictions = probability_model.predict(test_images)
    
pred_list = []
conf_list = []
w_conf_list = []
index_list = []
true_list = []
wrong_list = []
    
t_conf = 0
w_count = 0

for i in range(10000):
    predicted_label = np.argmax(predictions[i])
    confidence = round(100*np.max(predictions[i]), 2)
    t_conf += confidence
    pred_list.append(class_names[predicted_label])
    conf_list.append(confidence)
    if test_labels[i] != predicted_label:
        w_count += 1
        index_list.append(i)
        true_list.append(class_names[test_labels[i]])
        wrong_list.append(class_names[predicted_label])
        w_conf_list.append(confidence)

conf_mean = round(t_conf/10000, 2)

data_check['Predicted Label'] = pred_list
data_check['Confidence'] = conf_list
wrong = {'Index':index_list,'True Label':true_list,'Predicted Label':wrong_list,'Confidence':w_conf_list}
wrong_pred = pd.DataFrame(wrong)

##### Model Summary

In [ ]:
print('MODEL SUMMARY:\n')
print('Input Shape: ' + str(model.input_shape))
model.summary()

##### Validation

In [ ]:
val1 = round(train_acc[-1]*100, 2)
val2 = round(train_loss[-1]*100, 2)
val3 = round(valid_acc[-1]*100, 2)
val4 = round(valid_loss[-1]*100, 2)
val5 = round(test_acc*100, 2)
val6 = round(test_loss*100, 2)

print("Training Accuracy: " + str(val1))
print("Training Loss: " + str(val2))
print("Validation Accuracy: " + str(val3))
print("Validation Loss: " + str(val4))
print("Testing Accuracy: " + str(val5))
print("Testing Loss: " + str(val6))
#print("Confidence Mean: " + str(conf_mean))

In [ ]:
#vals = [val1, val2, val3, val4, val5, val6, conf_mean]
vals = [val1, val2, val3, val4, val5, val6]
#x = ['Training Accuracy','Training Loss','Validation Accuracy','Validation Loss',
     #'Testing Accuracy','Testing Loss','Confidence Mean']
x = ['Training Accuracy','Training Loss','Validation Accuracy','Validation Loss','Testing Accuracy','Testing Loss']
ind = np.arange(len(vals))
fig, ax = plt.subplots(figsize = (15,5))
fig.patch.set_facecolor('white')
ax.bar(ind,vals,width=0.7)
ax.set_title('SAMPLE TEST RESULTS\n\n'+params)
ax.set_ylabel('Average')
ax.set_ylim((0,100))
ax.set_xticks(range(len(vals)))
ax.set_xticklabels(x)
for i, v in enumerate(vals):
    plt.text(x=i-0.12, y =v+0.5 , s=f"{v}" , fontdict=dict(fontsize=12), fontweight='bold')
#plt.savefig("Results with Confidence", dpi=300)
#plt.savefig("Results", dpi=300)
plt.show()

##### Complete Prediction List

In [ ]:
print(data_check)

##### Wrong Predictions

In [ ]:
print('Wrong Predictions: ' + str(w_count))

In [ ]:
print(wrong_pred)

##### Save File

In [ ]:
#data_check.to_csv('prediction.csv', index=False)
#wrong_pred.to_csv('wrong_prediction.csv', index=False)

##### Sample Test

In [ ]:
def sample(ind):
    for r in range(10):
        print("  "+ class_names[r] + " = " + str(predictions[ind][r]*100))
    plt.figure(figsize=(10,4)).patch.set_facecolor('white')
    plt.subplot(1,2,1)
    plt.grid(False)
    plt.title('Test Image ' + str(ind))
    plt.xticks([])
    plt.yticks([])
    plt.imshow(test_images[i], cmap=plt.cm.binary)
    predicted_label = np.argmax(predictions[ind])   
    if predicted_label == test_labels[ind]:
        color = 'green'
    else:
        color = 'red' 
    confidence = 100*np.max(predictions[ind])
    plt.xlabel("{} {:2.0f}% ({})".format(class_names[predicted_label], confidence, class_names[test_labels[ind]]), color=color)
    plt.subplot(1,2,2)
    plt.grid(False)
    plt.title('Prediction Array')
    plt.xticks(range(10))
    plt.yticks([])
    thisplot = plt.bar(range(10), predictions[ind], color="#777777")
    plt.ylim([0, 1])
    thisplot[predicted_label].set_color('red')
    thisplot[test_labels[ind]].set_color('green')

In [ ]:
#Graphing the confidence of predictions
#Choose an image and predict
radnum = np.random.randint (0, 9999+1)
print("Sample Test data prediction: ")
sample(radnum)
#plt.savefig("Sample Prediction", dpi=300)
plt.show()

In [ ]:
random_num = random.choice(index_list)
print("Wrong Sample Test data prediction: ")
sample(random_num)
#plt.savefig("Wrong Sample Prediction", dpi=300)
plt.show()